# Plan-and-Solve Agent 深度解析与实战

Plan-and-Solve 是一种强大的智能体设计模式，它将复杂的任务解决过程解耦为两个明确的阶段：
1.  **规划 (Planning)**：理解问题，制定分步行动计划。
2.  **执行 (Execution)**：按部就班地执行计划，并在每一步利用上一步的结果。

这种模式特别适合解决长链条推理问题或多步骤任务。

## 1. 环境准备与基础类

首先，我们需要导入必要的库，并确保 LLM 客户端和 UI 工具已经准备就绪。

In [1]:
import ast # 用于解析 LLM 返回的 Python 列表
import json 
import os
import sys
from typing import List, Optional
from dotenv import load_dotenv

# 假设这些是我们项目中已经封装好的基础工具类
from llm_client import HelloAgentsLLM
from ui_utils import print_html

# 加载环境变量 (API Key)
load_dotenv()

# 初始化 LLM 客户端
llm_client = HelloAgentsLLM()

## 2. 规划器 (Planner)

**规划器**是大脑的战略中心。它的唯一职责是接收用户的问题，并将其分解为一系列逻辑严密、可执行的子步骤（Python 列表格式）。它不负责具体的计算或操作，只负责“想”。

### 核心实现

In [ ]:
PLANNER_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。你的任务是将用户提出的复杂问题分解成一个由多个简单步骤组成的行动计划。
请确保计划中的每个步骤都是一个独立的、可执行的子任务，并且严格按照逻辑顺序排列。
你的输出必须是一个Python列表，其中每个元素都是一个描述子任务的字符串。

问题: {question}

请严格按照以下格式输出你的计划，```python与```作为前后缀是必要的:
["步骤1", "步骤2", "步骤3", ... ]
"""

In [4]:
class Planner:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def _parse_plan(self, response_text: str) -> List[str]:
        """从 LLM 响应中解析计划列表"""
        try:
            # 尝试提取 markdown 代码块中的内容
            if "```python" in response_text:
                plan_str = response_text.split("```python")[1].split("```")[0].strip()
            elif "```" in response_text:
                plan_str = response_text.split("```")[1].split("```")[0].strip()
            else:
                plan_str = response_text.strip()
            
            # 使用 ast.literal_eval 安全地解析 Python 列表
            plan = ast.literal_eval(plan_str)
            
            if isinstance(plan, list):
                return plan
            return []
        except (ValueError, SyntaxError, IndexError):
            return []

    def plan(self, question: str) -> List[str]:
        """
        根据用户问题生成一个行动计划。
        """
        prompt = PLANNER_PROMPT_TEMPLATE.format(question=question)
        messages = [{"role": "user", "content": prompt}]
        
        print_html("正在生成计划...", title="🧠 Planner Thinking")
        response_text = self.llm_client.think(messages=messages) or ""
        
        # 解析计划
        plan = self._parse_plan(response_text)
        
        if plan:
            print_html(plan, title="✅ Plan Generated")
            return plan
        else:
            print_html(f"无法解析计划或生成格式错误。\n原始响应: {response_text}", title="❌ Plan Error")
            return []

## 3. 执行器 (Executor)

**执行器**是行动派。它拿到规划器生成的“剧本”（Plan），一步一步地去执行。
它的关键能力在于：在解决当前步骤时，它能看到**原始问题**、**完整计划**以及**之前的历史结果**。这使得它既专注当下，又拥有全局视野。

### 核心实现

In [5]:
EXECUTOR_PROMPT_TEMPLATE = """
你是一位顶级的AI执行专家。你的任务是严格按照给定的计划，一步步地解决问题。
你将收到原始问题、完整的计划、以及到目前为止已经完成的步骤和结果。
请你专注于解决“当前步骤”，并仅输出该步骤的最终答案，不要输出任何额外的解释或对话。

# 原始问题:
{question}

# 完整计划:
{plan}

# 历史步骤与结果:
{history}

# 当前步骤:
{current_step}

请仅输出针对“当前步骤”的回答:
"""

In [6]:
class Executor:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client

    def execute(self, question: str, plan: List[str]) -> str:
        history = ""
        final_answer = ""
        
        print_html("开始执行计划...", title="🚀 Execution Start")
        
        for i, step in enumerate(plan, 1):
            prompt = EXECUTOR_PROMPT_TEMPLATE.format(
                question=question, 
                plan=plan, 
                history=history if history else "无", 
                current_step=step
            )
            messages = [{"role": "user", "content": prompt}]
            
            response_text = self.llm_client.think(messages=messages) or ""
            
            # 渲染结果
            print_html(f"**Step**: {step}\n**Result**: {response_text}", title=f"Step {i}/{len(plan)} Result")
            
            history += f"步骤 {i}: {step}\n结果: {response_text}\n\n"
            final_answer = response_text
            
        return final_answer

## 4. 智能体整合 (Agent)

最后，我们将规划器和执行器封装在一个 Agent 类中，提供一键调用的 `run` 接口。

### 核心实现

In [7]:
class PlanAndSolveAgent:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client
        self.planner = Planner(self.llm_client)
        self.executor = Executor(self.llm_client)

    def run(self, question: str):
        print_html(question, title="🏁 Task Start")
        
        # 1. 规划
        plan = self.planner.plan(question)
        if not plan:
            print_html("无法生成有效的行动计划，任务终止。", title="🛑 Task Aborted")
            return
            
        # 2. 执行
        final_answer = self.executor.execute(question, plan)
        
        print_html(final_answer, title="🎉 Final Answer")

## 5. 实战演示

### 经典问题演示

现在，让我们用一个经典的数学应用题来测试我们的 Plan-and-Solve Agent。

**问题**：一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？

In [8]:
# 初始化智能体
agent = PlanAndSolveAgent(llm_client)

# 定义问题
question = "一个水果店周一卖出了15个苹果。周二卖出的苹果数量是周一的两倍。周三卖出的数量比周二少了5个。请问这三天总共卖出了多少个苹果？"

# 运行智能体
agent.run(question)

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


### 鲁棒性测试：异常处理

除了常规任务，我们还需要测试智能体在面对包含逻辑错误的问题时表现如何。
Plan-and-Solve 模式的一个潜在优势是，由于步骤是分步执行的，智能体有机会在中间步骤识别并处理错误，而不是让整个过程崩溃。

**测试场景**：我们将要求智能体执行一个包含“除以零”操作的任务。

**问题**：计算 10 除以 0 的结果，然后将结果加上 5。

**预期行为**：
1.  **Planner** 应该能正常分解步骤（除法 -> 加法）。
2.  **Executor** 在执行“除以零”这一步时，不应导致程序抛出 Python 异常而中断，而是应该由 LLM 识别出这是个数学谬误或无效操作，并以自然语言的形式报告错误（例如“除数不能为零”或返回 `None`/`Error`）。
3.  后续步骤应能根据前一步的错误结果做出合理的反应。

**观察点**：请注意观察 `Step` 执行结果。理想情况下，智能体会在执行除法步骤时指出错误，而不是胡乱给出一个数字，或者导致代码运行报错。

In [11]:
# 定义包含逻辑陷阱的问题
trick_question = "计算 10 除以 0 的结果，然后将结果加上 5。"

# 运行智能体进行测试
agent.run(trick_question)

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


### 测试分析与启示

通过“除以零”这个边界测试用例，我们观察到了当前 Plan-and-Solve 实现的一些有趣行为和局限性：

**观察结果**：
*   **Planner** 表现出了“编程思维”，它没有指出数学上的无效性，而是试图通过编程手段（如 `try-except`）来规避错误，生成了一个“捕获异常”的计划。
*   **Executor** 则表现为“盲目执行”，它机械地执行了计划，甚至将错误信息（Error Message）与数字 `5` 进行了无意义的字符串拼接，只为了给出一个“最终答案”。

**关键启示**：

1.  **“执行惯性”陷阱**：
    Plan-and-Solve 模式倾向于线性地完成任务。如果没有明确的指令让智能体在遇到逻辑谬误时“停下来”，它往往会尝试“强行”走完流程，导致产出荒谬的结果（例如 `error_message + 5`）。

2.  **角色定位的重要性**：
    当前的 Prompt 可能过于强调“执行步骤”和“输出结果”，导致 LLM 即使发现逻辑不对，也倾向于扮演一个“尽职尽责但不懂变通”的代码执行器，而不是一个具备批判性思维的逻辑推理者。

3.  **改进方向**：
    *   **增加反思机制**：在 Executor 中引入检查步骤，如果某一步结果包含“Error”或无意义，应主动中断任务。
    *   **优化 Prompt**：明确告诉智能体，面对无效输入或逻辑错误时，直接拒绝执行并返回原因，是比强行计算更好的选择。

这个测试提醒我们：**智能体不仅需要“会做事”（执行力），更需要“懂对错”（判断力）。**

# 进阶：Plan-and-Solve Pro (动态重规划)

针对基础版在异常处理上的短板，我们引入 **Replanning (动态重规划)** 机制。
Pro 版智能体不再是一条道走到黑，而是具备了自我反思和纠错的能力：
1.  **执行后评估 (Critic)**：每做一步，都由一个“裁判”来判断这一步是否成功。
2.  **动态重规划 (Replanning)**：如果遇到阻碍，根据错误原因重新生成后续计划。

## 1. 增强版规划器 (ReplanningPlanner)

除了生成初始计划，它现在还学会了 `replan`：当收到失败反馈时，基于已完成的历史和失败原因，重新规划剩余的路径。

In [12]:
import json
from typing import Tuple

# 新增：重规划 Prompt
REPLAN_PROMPT_TEMPLATE = """
你是一个顶级的AI规划专家。智能体在执行某个计划时遇到了阻碍，需要你协助进行动态重规划。

# 原始问题:
{question}

# 已完成的历史步骤与结果:
{history}

# 遇到的问题/阻碍:
{failure_reason}

你的任务是：基于目前的状况，为**剩余未完成的目标**生成一个新的行动计划。
请不要包含已经完成的步骤，只输出接下来的步骤。

请严格按照以下格式输出你的新计划，```python与```作为前后缀是必要的:
["新步骤1", "新步骤2", ...]
"""

In [13]:
class ReplanningPlanner(Planner): # 继承自基础 Planner
    def replan(self, question: str, history: str, failure_reason: str) -> List[str]:
        prompt = REPLAN_PROMPT_TEMPLATE.format(
            question=question,
            history=history,
            failure_reason=failure_reason
        )
        messages = [{"role": "user", "content": prompt}]
        
        print_html("正在进行动态重规划...", title="🔄 Replanning Thinking")
        response = self.llm_client.think(messages=messages) or ""
        
        return self._parse_plan(response)

## 2. 增强版执行器 (ReplanningExecutor)

Executor 现在拥有了一个 **Critic (评论家)** 功能。它使用 `EVAL_PROMPT_TEMPLATE` 来让 LLM 自我评估执行结果是否有效。

In [14]:
# 新增：评估 Prompt
EVAL_PROMPT_TEMPLATE = """
你是一个客观的裁判。请判断以下任务步骤的执行结果是否成功。

任务步骤: {step}
执行结果: {result}

判断标准：
1. 如果结果是一个明确的答案（即使答案是"无解"、"不存在"，只要是基于事实的合理结论），视为【成功】。
2. 如果结果表明发生了系统错误、工具调用失败、或者回答"我不知道"且没有给出理由，视为【失败】。
3. 特别注意：如果结果解释了为什么无法计算，这属于成功的回答。

请只输出 JSON 格式，格式如下：
```json
{{
    "success": true/false,
    "reason": "简短的理由"
}}
```
"""

In [15]:
class ReplanningExecutor(Executor): # 继承自基础 Executor
    def execute_step(self, question: str, plan: List[str], history: str, current_step: str) -> str:
        # 复用父类的 prompt 逻辑，但这里只执行单步
        prompt = EXECUTOR_PROMPT_TEMPLATE.format(
            question=question, 
            plan=plan, 
            history=history if history else "无", 
            current_step=current_step
        )
        messages = [{"role": "user", "content": prompt}]
        return self.llm_client.think(messages=messages) or ""

    def evaluate_result(self, step: str, result: str) -> Tuple[bool, str]:
        """使用 LLM (Critic) 进行语义评估"""
        prompt = EVAL_PROMPT_TEMPLATE.format(step=step, result=result)
        messages = [{"role": "user", "content": prompt}]
        
        response = self.llm_client.think(messages=messages) or "{}"
        
        try:
            # 简单的 JSON 提取逻辑
            json_str = response
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
                
            eval_result = json.loads(json_str)
            return eval_result.get("success", False), eval_result.get("reason", "Unknown reason")
        except Exception:
            return True, "Critic failed to parse, assuming success."

## 3. 增强版智能体 (ReplanningAgent)

主循环逻辑发生了质变：从 `for` 循环变成了 `while` 循环，以便随时动态修改计划队列。

In [16]:
class ReplanningAgent:
    def __init__(self, llm_client: HelloAgentsLLM):
        self.llm_client = llm_client
        self.planner = ReplanningPlanner(self.llm_client)
        self.executor = ReplanningExecutor(self.llm_client)

    def run(self, question: str):
        print_html(question, title="🏁 Task Start (Pro Agent)")
        
        # 1. 初始规划
        plan = self.planner.plan(question)
        if not plan: return

        history_str = ""
        final_answer = ""
        
        # 2. 动态执行循环
        while plan:
            current_step = plan.pop(0) # 取出队首步骤
            
            # 执行
            result = self.executor.execute_step(question, plan, history_str, current_step)
            print_html(f"**Step**: {current_step}\n**Result**: {result}", title="🎬 Executing Step")

            # 评估 (Critic)
            success, message = self.executor.evaluate_result(current_step, result)
            
            if success:
                # 成功：记录历史，继续下一步
                history_str += f"步骤: {current_step}\n结果: {result}\n\n"
                final_answer = result
            else:
                # 失败：触发重规划
                print_html(f"Critic 判定失败: {message}", title="⚠️ Execution Rejected")
                
                # 基于失败原因，生成新的后续计划
                new_remaining_plan = self.planner.replan(question, history_str, failure_reason=message)
                
                if new_remaining_plan:
                    plan = new_remaining_plan # 替换旧的剩余计划
                    print_html(f"已更新后续计划: {plan}", title="🔄 Plan Updated")
                else:
                    print_html("重规划失败，任务终止。", title="🛑 Stop")
                    return

        print_html(final_answer, title="🎉 Final Answer")

## 4. 对比测试：再次挑战除以零

现在，让我们用刚才那个失败的例子来考验一下 **Pro Agent**。

In [22]:
# 初始化 Pro 智能体
pro_agent = ReplanningAgent(llm_client)

# 再次运行相同的测试
pro_agent.run("计算 10 除以 0 的结果，然后将结果加上 5。")

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


**预期观察**：
1.  当 Executor 试图执行“除以零”并返回某种错误描述时。
2.  **Critic** 应该敏锐地指出这一步“失败”了（或者虽然算不出数，但作为对“除以零”的响应是合理的）。
3.  如果 Critic 判定为失败，Planner 可能会介入，修改后续计划（例如取消“加上 5”的步骤），从而给出一个更符合逻辑的最终回答。

In [17]:
# 初始化 Pro 智能体
pro_agent = ReplanningAgent(llm_client)

# 再次运行相同的测试
pro_agent.run("计算 10 除以 0 的结果，然后将结果加上 5。")

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


## 5. 进阶挑战：解决自然语言的多义性

在多次测试中，我们发现了一个有趣的现象：
当执行结果为 `"undefined"` 或 `"error"` 时，**Critic** 有时会将其误判为【失败】，触发不必要的重规划，导致任务陷入死循环或报错。

**原因分析**：
自然语言具有多义性。词语 `"undefined"` 可能意味着：
1.  **数学逻辑层面**：除以零无意义（这是对任务的**正确响应**，应判定为【成功】）。
2.  **系统程序层面**：变量未定义或代码崩溃（这是执行故障，应判定为【失败】）。

如果 Prompt 不够清晰，LLM 很容易混淆这两者。

### 优化方案：消除歧义的 Prompt

我们需要修改 `EVAL_PROMPT_TEMPLATE`，明确告诉 Critic：**如果结果指出了任务在逻辑上的不可行性（如数学陷阱），这属于成功的执行。**

In [18]:
# --- 优化后的 Critic Prompt ---
EVAL_PROMPT_TEMPLATE_V2 = """
你是一个客观的裁判。请判断以下任务步骤的执行结果是否成功。

任务步骤: {step}
执行结果: {result}

判断标准：
1. 【成功】(Success):
   - 得到了明确的计算结果或信息。
   - **关键修正**：如果结果明确指出了任务在逻辑上无法完成（例如“除数不能为零”、“查询无结果”、“undefined due to math logic”），这属于对任务的有效响应，应视为成功。
   - 只要 LLM 给出了基于事实的理由来解释为什么无法继续，都算成功。

2. 【失败】(Failure):
   - 真正的系统级错误（如 "Timeout", "Connection Refused", "Python Syntax Error"）。
   - LLM 拒绝回答但未给出理由（如 "I don't know"）。
   - 结果与任务步骤完全无关。

请只输出 JSON 格式，格式如下：
```json
{{
    "success": true/false,
    "reason": "简短的理由"
}}
```
"""

In [19]:
# 更新 Executor 使用新的 Prompt
class OptimizedReplanningExecutor(ReplanningExecutor):
    def evaluate_result(self, step: str, result: str) -> Tuple[bool, str]:
        # 使用 V2 版本的 Prompt
        prompt = EVAL_PROMPT_TEMPLATE_V2.format(step=step, result=result)
        messages = [{"role": "user", "content": prompt}]
        
        response = self.llm_client.think(messages=messages) or "{}"
        
        try:
            json_str = response
            if "```json" in response:
                json_str = response.split("```json")[1].split("```")[0].strip()
            elif "```" in response:
                json_str = response.split("```")[1].split("```")[0].strip()
                
            eval_result = json.loads(json_str)
            return eval_result.get("success", False), eval_result.get("reason", "Unknown reason")
        except Exception:
            return True, "Critic failed to parse, assuming success."

# 更新 Agent 使用优化后的 Executor
class OptimizedReplanningAgent(ReplanningAgent):
    def __init__(self, llm_client: HelloAgentsLLM):
        super().__init__(llm_client)
        # 替换为优化后的 Executor
        self.executor = OptimizedReplanningExecutor(self.llm_client)

### 再次验证

现在，让我们使用优化后的 Agent 再次运行“除以零”测试。
这一次，即使 Executor 返回了 `"undefined"`，Critic 也应该能识别出这是因为“除数不能为零”，从而判定为**成功**，并允许流程正常结束（而不是触发错误的重规划）。

**预期结果**：
Critic 应该会给出类似这样的评价：
> *"Success: True. Reason: The result correctly identifies that division by zero is undefined."*

从而避免了之前的误判。这展示了 Prompt Engineering 在消除 LLM 不确定性方面的关键作用。

In [20]:
# 初始化优化后的智能体
optimized_agent = OptimizedReplanningAgent(llm_client)

print("--- 开始测试优化后的 Agent ---")
optimized_agent.run("计算 10 除以 0 的结果，然后将结果加上 5。")

--- 开始测试优化后的 Agent ---


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


...

🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!
🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


🧠 正在调用 mimo-v2-flash 模型...
✅ 大语言模型响应成功!


## 6. 总结与反思：从执行者到思考者

通过本次实战演练，我们见证了一个智能体从“线性执行”到“动态反思”的进化过程。
特别是最后的“除以零”测试，完美展示了 **Replanning（重规划）** 机制的核心价值。

### 6.1 测试结果对比

| 智能体版本 | 面对“除以零”的表现 | 结果分析 |
| :--- | :--- | :--- |
| **基础版 (Plan-and-Solve)** | 机械执行，尝试将错误信息与数字相加 (`Error + 5`)。 | **失败**。缺乏判断力，为了执行而执行，产出无意义结果。 |
| **Pro 版 (未优化 Prompt)** | 能够识别错误，但 Critic 容易因歧义误判 `undefined` 为系统故障。 | **不稳定**。虽然拦截了错误，但处理过程较为生硬，可能触发不必要的重试。 |
| **Pro 版 (优化 Prompt)** | **完美自愈**。Critic 拒绝了含糊的 `undefined`，逼迫 Planner 生成解释性计划。 | **成功**。不仅避免了崩溃，还通过重规划输出了包含数学原理的详尽解释。 |

### 6.2 关键洞察：成功的“失败”

在最后的测试中，Critic 虽然判定第一步执行【失败】，但这恰恰是成功的关键：
1.  **Critic 的严格把关**：它指出单纯的 `undefined` 不足以作为答案，要求给出解释。
2.  **Planner 的智能响应**：收到反馈后，Planner 没有盲目重试，而是生成了一个**侧重于分析和解释**的新计划。
3.  **最终结果的质变**：智能体最终输出的不再是一个冷冰冰的报错，而是一段有理有据的分析：“因为除数不能为零，所以无法计算”。

### 6.3 核心代码回顾

整个系统的灵魂在于以下闭环逻辑：

```python
# 伪代码总结
while plan:
    step = plan.pop()
    result = execute(step)
    
    # 关键点：每一步都要经过裁判的审视
    success, reason = critic.evaluate(step, result)
    
    if not success:
        # 关键点：遇到挫折，不是报错，而是重规划
        new_plan = planner.replan(failure_reason=reason)
      

### 6.4 结语

Plan-and-Solve Pro 模式证明了：**智能体的强大不仅在于它能做什么（Tool Use），更在于它知道自己做不到什么，并能据此调整策略（Reflection & Replanning）。**

这种“执行-评估-修正”的闭环，是构建高鲁棒性 AI Agent 的必经之路。